In [181]:
import pandas as pd
import numpy as np
import datetime
import talib
import pdb
import quantstats as qs
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import colors

In [182]:
data = pd.read_csv('D:/Markets & markets/Python/Data Bank/3 Min/DP_HDFC.csv')
data

,Date,Time,Open,High,Low,Close,Volume
0,02-02-2015,09:15,1262.50,1267.95,1253.90,1255.00,120331
1,02-02-2015,09:18,1255.00,1256.40,1254.80,1255.35,29919
2,02-02-2015,09:21,1255.35,1256.00,1249.50,1251.05,37585
3,02-02-2015,09:24,1251.05,1251.95,1248.45,1249.70,35324
4,02-02-2015,09:27,1249.65,1250.00,1248.50,1249.85,24305
...,...,...,...,...,...,...,...
198673,14-07-2021,15:15,2533.35,2535.10,2531.25,2531.30,76818
198674,14-07-2021,15:18,2531.45,2532.95,2525.00,2525.00,99699
198675,14-07-2021,15:21,2525.10,2527.35,2525.00,2525.55,104470
198676,14-07-2021,15:24,2525.60,2527.25,2525.05,2527.25,84874


In [183]:
# For 3min timeframe

last_candle = data.iloc[-1]
entry_start_time = data.iloc[5]['Time']
entry_stop_time = data.iloc[105]['Time']
exit_time = data.iloc[119]['Time']

In [184]:
#Indicator Calculations

data['Entry_level'] = data['High'].shift(1).rolling(window = 55).max()
data['Exit_level'] = data['Low'].shift(0).rolling(window = 22).min()
data['RSI'] = talib.RSI(data['Close'], timeperiod=20)
data['CL'] = ((data['High'].shift(0).rolling(window =18).max())+(data['Low'].shift(0).rolling(window =18).min()))/2
data['BL'] = ((data['High'].shift(0).rolling(window =52).max())+(data['Low'].shift(0).rolling(window =52).min()))/2
data['LSA'] = (data['CL'].shift(52) + data['BL'].shift(52))/2
data['LSB'] = (((data['High'].shift(0).rolling(window =104).max())+(data['Low'].shift(0).rolling(window =104).min()))/2).shift(52)
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod = 14)
ATR_upper_limit = data['ATR'].mean()+3*data['ATR'].std()

In [185]:
#Trade Parameters

Sl_value = 0.01
Profit_Target = 0.05 
data['stoploss'] = data['Close'] * (1-Sl_value)
data['target'] = data['Close'] * (1+Profit_Target)


In [186]:
# Position Sizing

risk = 5000
ATR = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=4)
data['Quantity'] = round(risk/(ATR*2),0)

In [187]:
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
trade_log = pd.DataFrame(columns='trade_type,entry_date,entry_time,entry_price,Quantity,exit_time,exit_price'.split(','))
data

,Date,Time,Open,High,Low,Close,Volume,Entry_level,Exit_level,RSI,CL,BL,LSA,LSB,ATR,stoploss,target,Quantity
0,03-02-2015,10:45,1256.00,1256.15,1252.65,1253.65,12656,1277.95,1251.25,43.201707,1254.475,1263.675,1268.5125,1264.050,2.865267,1241.1135,1316.3325,944.0
1,03-02-2015,10:48,1254.30,1255.85,1253.15,1255.50,9465,1276.10,1251.25,46.793815,1254.475,1263.675,1268.6750,1264.050,2.853462,1242.9450,1318.2750,939.0
2,03-02-2015,10:51,1255.75,1257.90,1255.05,1257.00,12060,1276.10,1251.25,49.518655,1254.575,1263.675,1268.6750,1264.050,2.853215,1244.4300,1319.8500,923.0
3,03-02-2015,10:54,1257.00,1257.65,1254.25,1254.50,11463,1276.10,1251.25,45.436333,1254.575,1263.675,1268.6750,1264.050,2.892271,1241.9550,1317.2250,868.0
4,03-02-2015,10:57,1254.50,1257.00,1254.00,1256.55,13197,1276.10,1251.25,49.061086,1254.575,1263.675,1269.2375,1264.050,2.899966,1243.9845,1319.3775,859.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198518,14-07-2021,15:15,2533.35,2535.10,2531.25,2531.30,76818,2543.85,2528.00,46.034901,2532.425,2535.925,2531.1750,2532.975,2.021361,2505.9870,2657.8650,906.0
198519,14-07-2021,15:18,2531.45,2532.95,2525.00,2525.00,99699,2543.85,2525.00,37.197755,2530.925,2534.425,2531.1750,2532.975,2.444835,2499.7500,2651.2500,616.0
198520,14-07-2021,15:21,2525.10,2527.35,2525.00,2525.55,104470,2543.85,2525.00,38.286445,2530.925,2534.425,2531.4750,2532.975,2.438061,2500.2945,2651.8275,689.0
198521,14-07-2021,15:24,2525.60,2527.25,2525.05,2527.25,84874,2543.85,2525.00,41.581350,2530.925,2534.425,2531.7000,2532.975,2.421057,2501.9775,2653.6125,764.0


In [ ]:
long_trade_triggered = 0
for index, row in data[1:].iterrows():
    previous_row = data.iloc[index-1]
    previous_row2 = data.iloc[index-26]
    if ((previous_row['Close'] > previous_row['LSA']) &
        (previous_row['Close'] > previous_row['LSB']) &
        (previous_row['Close'] > previous_row['CL']) &
        (previous_row['Close'] > previous_row['BL']) &
        (previous_row['CL'] > previous_row['BL']) &
        (previous_row['Close'] > previous_row['Entry_level']) &  
        (previous_row['RSI'] > 60) & (previous_row['RSI'] < 80) &
        (previous_row['ATR'] < ATR_upper_limit) &  
        (previous_row['Time'] > entry_start_time ) & 
        (previous_row['Time'] <= entry_stop_time ) & 
        (long_trade_triggered == 0)):
        long_trade_triggered = 1
        trade_log = trade_log.append({'trade_type':'long',
                                      'entry_date':row['Date'],
                                      'entry_time':row['Time'],
                                      'entry_price':row['Open'],
                                      'Quantity':previous_row['Quantity']},ignore_index=True)


    elif ((long_trade_triggered == 1) & 
         ((previous_row['Close'] < previous_row['Exit_level']) or
          (previous_row['Close'] > previous_row['target']) or
          (previous_row['Close'] < previous_row['stoploss']) or
          (previous_row['Time'] == exit_time))):

        long_trade_triggered = 0
        trade_log = trade_log.append({'exit_time':row['Time'],
                                      'exit_price':row['Open']},ignore_index=True)

trade_log['exit_time'] = trade_log['exit_time'].shift(-1)
trade_log['exit_price'] = trade_log['exit_price'].shift(-1)

trade_log.dropna(inplace=True)
trade_log.reset_index(drop=True,inplace=True)

In [ ]:
rcParams['figure.figsize'] = 18,12
trade_log['Pnl'] = (trade_log['exit_price'] - trade_log['entry_price'])
trade_log['Pnl'].cumsum().plot()

In [ ]:
trade_log['MTM'] = trade_log['Pnl'] * trade_log['Quantity']
trade_log['MTM_cumsum'] = trade_log['MTM'].cumsum()
trade_log['Return'] = round((trade_log['Pnl']/trade_log['entry_price'])*100,2)
trade_log['Drawdown'] = trade_log['MTM_cumsum'] - trade_log['MTM_cumsum'].cummax()
max_drawdown = round(trade_log['Drawdown'].min(),2)
trade_log['Drawdown'].plot()
equity_High = round(trade_log['MTM_cumsum'].max(),3)
Calmar_ratio = round((trade_log['MTM_cumsum'].max()/abs(max_drawdown)),3)
Number_of_trades =trade_log.shape[0]
Average_per_trade = round(equity_High/Number_of_trades,2)
trade_log_Average = trade_log['MTM_cumsum']
trade_log

In [ ]:
qs.extend_pandas()
win_rate = round(qs.stats.win_rate(trade_log['MTM'])*100,3)
Average_Loss = round(qs.stats.avg_loss(trade_log['MTM']),3)
Average_win = round(qs.stats.avg_win(trade_log['MTM']),3)
reward_risk = round(abs(Average_win/Average_Loss),3)
max_loosing_streak = qs.stats.consecutive_losses(trade_log['MTM'])
max_winning_streak = qs.stats.consecutive_wins(trade_log['MTM'])
kelly_criterion = round(qs.stats.kelly_criterion(trade_log['MTM']),3)
kurtosis = round(qs.stats.kurtosis(trade_log['MTM']),3)
value_at_risk = round(qs.stats.value_at_risk(trade_log['MTM']),3)
best_case = round(qs.stats.best(trade_log['MTM']),3)
worst_case = round(qs.stats.worst(trade_log['MTM']),3)
Expectancy = round(((win_rate/100) * reward_risk)-(1-(win_rate/100)),2)

print("Number of trades        =  " + str(Number_of_trades))
print("win_rate in %age        =  " + str(win_rate))
print("Average Per trade       =  " + str(Average_per_trade))
print("Average_Loss            =  " + str(Average_Loss))
print("Average_win             =  " + str(Average_win))
print("reward_risk             =  " + str(reward_risk))
print("max_loosing_streak      =  " + str(max_loosing_streak))
print("max_winning_streak      =  " + str(max_winning_streak))
print("kelly_criterion         =  " + str(kelly_criterion))
print("kurtosis                =  " + str(kurtosis))
print("value_at_risk           =  " + str(value_at_risk))
print("best_case               =  " + str(best_case))
print("worst_case              =  " + str(worst_case))
print("equity_High             =  " + str(equity_High))
print("Max_Drawdown            =  " + str(max_drawdown))
print("Calmar_ratio            =  " + str(Calmar_ratio))
print("Expectancy              =  " + str(Expectancy))

In [ ]:
trade_log.count()